# CreditOne Classification 
Class 5 Task 3

Digging into SVM KNN RF

#### Import modules

In [1]:
#enable auto complete
%config IPCompleter.greedy=True

In [2]:
%matplotlib inline

In [3]:
#imports
#numpy,pandas,scipy,math,matplotlib
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt

#estimators
#svm
from sklearn import svm
#knn
from sklearn.neighbors import KNeighborsClassifier
#RF
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

#model metrics
from sklearn.model_selection import cross_val_score
import sklearn.metrics as skm

#cross validation
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import VarianceThreshold


#### import data

In [4]:
#data
rawData = pd.read_csv('default of credit card clients.csv', header=1)
rawData.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,4810,500000,1,3,3,53,0,0,0,0,...,125218,126650,114473,12602,11000,4711,6000,11000,10000,0
1,15096,500000,1,1,3,38,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,1
2,81,470000,2,3,3,33,0,0,0,0,...,69923,29271,29889,6400,7566,3000,960,1000,3000,0
3,14889,450000,2,1,3,37,1,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
4,11072,450000,1,1,3,37,-2,-2,-2,-2,...,317,1614,324,303,5041,1,1622,325,316,0


In [5]:
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null int64
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6               

In [6]:
df = rawData

In [7]:
#change to categorical

df.SEX = df.SEX.astype('category')
df.MARRIAGE = df.MARRIAGE.astype('category')
df.AGE = df.AGE.astype('category')
df.EDUCATION = df.EDUCATION.astype('category')

In [8]:
#bin Age data
df['AGE'] = pd.cut(df['AGE'], [20,30,40,50,60,70,80],labels=['20-30','30-40','40-50', '50-60', '60-70','80-70'])

In [9]:
#dummiefy age
AGE_dummies = pd.get_dummies(df['AGE'],prefix = "AGE", columns=None, sparse=False, drop_first=False)

In [10]:
AGE_dummies

,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,AGE_80-70
0,0,0,0,1,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
5,0,1,0,0,0,0
6,0,0,0,1,0,0
7,0,0,1,0,0,0
8,0,1,0,0,0,0
9,0,1,0,0,0,0


In [11]:
AGE_dummies

,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,AGE_80-70
0,0,0,0,1,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
5,0,1,0,0,0,0
6,0,0,0,1,0,0
7,0,0,1,0,0,0
8,0,1,0,0,0,0
9,0,1,0,0,0,0


In [12]:
#add age dummies to df and delete age
df=df.join(AGE_dummies)
del df['AGE']

In [13]:
#move default column to end

#make an array of column names
cols = df.columns.tolist()

In [14]:
cols

['ID',
 'LIMIT_BAL',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'default payment next month',
 'AGE_20-30',
 'AGE_30-40',
 'AGE_40-50',
 'AGE_50-60',
 'AGE_60-70',
 'AGE_80-70']

In [15]:
#move default to the end
cols.insert(-1, cols.pop(cols.index('default payment next month')))
cols
df=df[cols]

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 30 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null category
EDUCATION                     30000 non-null category
MARRIAGE                      30000 non-null category
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6                     30000 non-null int64
PAY_AMT1       

In [17]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,default payment next month,AGE_80-70
0,4810,500000,1,3,3,0,0,0,0,0,...,6000,11000,10000,0,0,0,1,0,0,0
1,15096,500000,1,1,3,-2,-2,-2,-2,-2,...,0,0,0,0,1,0,0,0,1,0
2,81,470000,2,3,3,0,0,0,0,0,...,960,1000,3000,0,1,0,0,0,0,0
3,14889,450000,2,1,3,1,-2,-2,-2,-2,...,0,0,0,0,1,0,0,0,0,0
4,11072,450000,1,1,3,-2,-2,-2,-2,-2,...,1622,325,316,0,1,0,0,0,0,0


In [18]:
df.shape

(30000, 30)

### Select Features


#### Remove low variance features

In [19]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(df)

array([[  4.81000000e+03,   5.00000000e+05,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.50960000e+04,   5.00000000e+05,   1.00000000e+00, ...,
          1.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  8.10000000e+01,   4.70000000e+05,   2.00000000e+00, ...,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  1.65820000e+04,   2.00000000e+04,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.15600000e+04,   2.00000000e+04,   2.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  6.39000000e+03,   1.00000000e+04,   1.00000000e+00, ...,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [20]:
df.shape

(30000, 30)

## Correlation Matrix

In [21]:
c = df.corr().abs()

In [22]:
s = c.unstack()
so = s.sort_values(kind="quicksort")

print (so)

#corrMat = df.corr().abs
#print(corrMat)

PAY_AMT4                    AGE_80-70                     0.000081
AGE_80-70                   PAY_AMT4                      0.000081
                            PAY_AMT5                      0.000103
PAY_AMT5                    AGE_80-70                     0.000103
PAY_AMT3                    AGE_80-70                     0.000422
AGE_80-70                   PAY_AMT3                      0.000422
                            PAY_6                         0.000475
PAY_6                       AGE_80-70                     0.000475
ID                          PAY_AMT5                      0.000652
PAY_AMT5                    ID                            0.000652
AGE_50-60                   ID                            0.000923
ID                          AGE_50-60                     0.000923
PAY_3                       PAY_AMT1                      0.001295
PAY_AMT1                    PAY_3                         0.001295
BILL_AMT4                   AGE_50-60                     0.00

In [23]:
del df['BILL_AMT1']
del df['BILL_AMT2']
del df['BILL_AMT3']
del df['BILL_AMT4']
del df['BILL_AMT5']


In [24]:
del df['PAY_0']
del df['PAY_2']
del df['PAY_3']
del df['PAY_4']
del df['PAY_5']


In [25]:
df.shape


(30000, 20)

In [26]:
#features
features = df.iloc[:,1:19]
print('Summary of feature sample')
features.head()

Summary of feature sample


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,PAY_6,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,default payment next month
0,500000,1,3,3,0,114473,12602,11000,4711,6000,11000,10000,0,0,0,1,0,0
1,500000,1,1,3,-2,0,0,0,0,0,0,0,0,1,0,0,0,1
2,470000,2,3,3,0,29889,6400,7566,3000,960,1000,3000,0,1,0,0,0,0
3,450000,2,1,3,-2,0,0,0,0,0,0,0,0,1,0,0,0,0
4,450000,1,1,3,-2,324,303,5041,1,1622,325,316,0,1,0,0,0,0


#### Select Dependent Variable

In [27]:
#dependent variable
depVar = df['default payment next month']

In [28]:
depVar.describe()

count    30000.000000
mean         0.221200
std          0.415062
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: default payment next month, dtype: float64

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 20 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null category
EDUCATION                     30000 non-null category
MARRIAGE                      30000 non-null category
PAY_6                         30000 non-null int64
BILL_AMT6                     30000 non-null int64
PAY_AMT1                      30000 non-null int64
PAY_AMT2                      30000 non-null int64
PAY_AMT3                      30000 non-null int64
PAY_AMT4                      30000 non-null int64
PAY_AMT5                      30000 non-null int64
PAY_AMT6                      30000 non-null int64
AGE_20-30                     30000 non-null uint8
AGE_30-40                     30000 non-null uint8
AGE_40-50                     30000 non-null uint8
AGE_50-60                     30000 non-null uint8
AGE_60-70      

### Establish Training Data


In [30]:
#run split
X_train, X_test, y_train, y_test = train_test_split(features, depVar, test_size = 0.30, random_state =123)
print("X_train",X_train.shape,'\n',"X_test", X_test.shape,'\n',"y_train", y_train.shape,'\n',"y_test", y_test.shape)

X_train (21000, 18) 
 X_test (9000, 18) 
 y_train (21000,) 
 y_test (9000,)


# Random Forest

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

RandomForestClassifier(n_estimators=10, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)

In [32]:

print(__doc__)

import numpy as np

from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

# get some data
X, y = X_test, y_test

# build a classifier
clf = RandomForestClassifier(n_estimators=20)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {"max_depth": 2,
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

Automatically created module for IPython interactive environment
RandomizedSearchCV took 6.15 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 1.000 (std: 0.000)
Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'max_features': 10, 'min_samples_leaf': 8, 'min_samples_split': 2}

Model with rank: 1
Mean validation score: 1.000 (std: 0.000)
Parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 3, 'max_features': 9, 'min_samples_leaf': 4, 'min_samples_split': 3}

Model with rank: 1
Mean validation score: 1.000 (std: 0.000)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 3, 'max_features': 9, 'min_samples_leaf': 6, 'min_samples_split': 10}

Model with rank: 1
Mean validation score: 1.000 (std: 0.000)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 6, 'min_samples_leaf': 6, 'min_samples_split': 9}

Model with rank: 1
Mean validation score: 1.000 (std: 0.00

ValueError: Parameter values for parameter (max_depth) need to be a sequence(but not a string) or np.ndarray.

In [91]:
RFclf = RandomForestClassifier(max_depth=4, random_state=0)

In [92]:
RFclf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [93]:
RFclf_pred = RFclf.predict(X_test)

In [94]:
print(" ***Random Forest***" ,'\n',"accuracy",skm.accuracy_score(y_test, RFclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, RFclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, RFclf_pred),'\n','Precision Score',skm.precision_score(y_test,RFclf_pred))

 ***Random Forest*** 
 accuracy 0.971444444444 
 average_precision_score 0.900376004433 
 Kappa 0.913589305248 
 Precision Score 1.0


In [95]:
print(RFclf.feature_importances_)

[  1.61198324e-02   6.85405903e-04   9.68943479e-04   1.25374074e-03
   1.04045748e-01   5.53290150e-03   1.69080764e-02   1.89982896e-02
   9.05778105e-03   1.89357508e-02   9.58467496e-03   4.02380099e-03
   8.60904483e-05   0.00000000e+00   0.00000000e+00   2.09164333e-04
   0.00000000e+00   7.93589800e-01]


In [96]:
confusion_RF = skm.confusion_matrix(y_test, RFclf_pred)
print(confusion_RF)
# 0: negative class (doesn't have default status)
# 1: positive class (has default status)


#        Pr_0  Pr_1
# Act_0   TN    FP 
# Act_1   FN    TP

[[6995    0]
 [ 257 1748]]


In [97]:
print(cross_val_score(RFclf, X_train, y_train))
RFclf.score(X_train, y_train)

[ 0.96414798  0.98571429  0.9815688 ]


0.9744761904761905

In [98]:
# Feature importance

print(RFclf.feature_importances_)

[  1.61198324e-02   6.85405903e-04   9.68943479e-04   1.25374074e-03
   1.04045748e-01   5.53290150e-03   1.69080764e-02   1.89982896e-02
   9.05778105e-03   1.89357508e-02   9.58467496e-03   4.02380099e-03
   8.60904483e-05   0.00000000e+00   0.00000000e+00   2.09164333e-04
   0.00000000e+00   7.93589800e-01]


In [99]:
RFclf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 4,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}